#### Install necesaary library

In [1]:
#!python -m spacy download en_core_web_sm

#### Import the necessary libraries

In [2]:
import spacy
from sklearn.model_selection import train_test_split
import pycrfsuite
import sys
import os
from nltk.tokenize import word_tokenize

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [5]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

In [6]:
def tuples_maker(inp, out):
    sample_tuple=[tuple(zipped) for zipped in zip(inp, out)]
    sample_data=[]
    for x in sample_tuple:
        sample_data.append([tuple(zipped) for zipped in zip(x[0], x[1])])
    sample_data
    return sample_data

In [7]:
tuples_maker(input_sigs, output_labels)

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [8]:
def triples_maker(whole_data):
    temp_list1=[]
    sample_data=[]
    for words in whole_data:
        temp_list2=[]
        for sigs in words:
            token=nlp(sigs[0])
            tag=token[0].tag_
            temp_list=list(sigs) #convert tuple to list for inserting tag
            temp_list.insert(1,tag) #insert tag at 1 position
            temp_list2.append(tuple(temp_list)) #convert list back to tuple
        sample_data.append(temp_list2)
    sample_data
    return sample_data

In [9]:
whole_data=tuples_maker(input_sigs, output_labels)

sample_data= triples_maker(whole_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'VB', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'LS', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'LS', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [10]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [11]:
def get_features(doc):
    return [token_to_features(doc,i) for i in range(len(doc))]


def get_labels(doc):
    return [label for token, pos, label in doc]

X=[get_features(doc)for doc in sample_data]
y=[get_labels(doc) for doc in sample_data]
   
x_train, x_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=42 )
#x_train, =train_test_split(X, y, )

### Training the CRF model with the features extracted using the feature extractor method

In [12]:
trainer=pycrfsuite.Trainer()

for xseq, yseq in zip(x_train,y_train):
    trainer.append(xseq,yseq)

trainer.set_params({'max_iterations':1000, 'c1':0.1, 'c2':0.01})

trainer.train('prescription_parser.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 868
Seconds required: 0.002

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 511.079592
Feature norm: 1.000000
Error norm: 106.622922
Active features: 853
Line search trials: 1
Line search step: 0.007007
Seconds required for this iteration: 0.001

***** Iteration #2 *****
Loss: 348.901040
Feature norm: 5.060515
Error norm: 137.412934
Active features: 832
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.002

***** Iteration #3 *****
Loss: 251.880359
Feature norm: 6.022915
Error norm: 59.373048
Active features: 857
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.0

### Predicting the test data with the built model

In [13]:
tagger=pycrfsuite.Tagger()
tagger.open('prescription_parser.model')

In [14]:
predictions=[tagger.tag(data) for data in x_test]

In [15]:
predictions 

[['FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['EVERY', 'Period', 'PeriodUnit'],
 ['QID'],
 ['Method', 'Qty', 'Form', 'Frequency', 'DAILY'],
 ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['EVERY', 'Period', 'PeriodUnit'],
 ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'],
 ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'],
 ['Method',
  'Qty',
  'Form',
  'Frequency',
  'PeriodUnit',
  'FOR',
  'Duration',
  'DurationUnit',
  'AT',
  'WHEN'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['QID'],
 ['Qty', 'Form', 'Qty', 'Form', 'Frequency'],
 ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'],
 ['EVERY', 'Period', 'PeriodUnit']]

### Putting all the prediction logic inside a predict method

In [16]:
def train_model(input_sigs, output_labels):
    #training entire dataset
    whole_data=tuples_maker(input_sigs, output_labels)
    sample_data= triples_maker(whole_data)
    
    X=[get_features(doc)for doc in sample_data]
    y=[get_labels(doc) for doc in sample_data]

    trainer=pycrfsuite.Trainer()

    for xseq, yseq in zip(X,y):
        trainer.append(xseq,yseq)

    trainer.set_params({'max_iterations':1000, 'c1':0.1, 'c2':0.01})

    #to prevent printing unneccessary output lines: training steps
    old_stdout = sys.stdout #extra code
    sys.stdout = open(os.devnull, "w") #extra code
    trainer.train('prescription_parser.model')
    sys.stdout = old_stdout #extra code

In [17]:
def predict(sig):
    
    tagger=pycrfsuite.Tagger()
    tagger.open('prescription_parser.model')

    nltest=nlp(sig) #converting text to nlptext for easy tokenization
    test_list=[]
    for token in nltest:
        test_tuple=(str(token), str(token.tag_))
        test_list.append(test_tuple)
    test_list=[test_list] #because the function token_to_features() needs list of lists

    test_data=[get_features(doc) for doc in test_list]

    tagger=pycrfsuite.Tagger()
    tagger.open('prescription_parser.model')

    predictions=[tagger.tag(data) for data in test_data]
    
    print(sig)
    print(predictions)
    
    return predictions

In [18]:
train_model(input_sigs, output_labels)

### Sample predictions

In [19]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [29]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'PeriodUnit', 'AT', 'WHEN']]


In [30]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [31]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]


In [32]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [33]:
predictions = predict("x 30 days")

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [34]:
predictions = predict("x 30 days")

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [35]:
predictions = predict("x 20 months")

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [36]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [37]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [38]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Frequency', 'TID']]


In [39]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']]


In [40]:
predictions = predict("inject 3 units orally")

inject 3 units orally
[['Method', 'Qty', 'Form', 'PO']]


In [41]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
[['PO', 'Method', 'Qty', 'Form', 'TID']]


In [42]:
predictions = predict("by mouth take three caps")

by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [43]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [44]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [45]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'BID', 'PO', 'AT', 'WHEN']]


In [46]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [47]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
[['FOR', 'Duration', 'DurationUnit', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [48]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
[['Qty', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'AND', 'WHEN', 'Period', 'PeriodUnit']]


In [49]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
[['FOR', 'Duration', 'Qty', 'Form']]


In [50]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']]


In [51]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [52]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [53]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
[['Method', 'Qty', 'Form', 'Qty', 'Form']]


In [54]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [55]:
prediction = predict("take two to four tabs")

take two to four tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [56]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']]


In [57]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [49]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [58]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [59]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [60]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [61]:
prediction = predict("every 60 mins")

every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [62]:
prediction = predict("every 10 mins")

every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [63]:
prediction = predict("every two to four months")

every two to four months
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [64]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [65]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [66]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
[['FOR', 'Duration', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [67]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit']]


In [68]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
[['PO', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'FOR', 'Duration', 'DurationUnit']]


In [69]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [70]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [71]:
prediction = predict("10 tabs")

10 tabs
[['Qty', 'Form']]
